In [7]:
import sys
import os

# Ruta al proyecto raíz donde está la carpeta 'scripts'
project_root = os.path.abspath('../../')  # o usa la ruta absoluta si prefieres
sys.path.append(project_root)

In [8]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np

from sklearn.metrics import roc_curve, RocCurveDisplay, roc_auc_score, \
                            auc, confusion_matrix, accuracy_score, \
                            classification_report

from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

import joblib

In [9]:
df_data_general = pd.read_csv('../../data/data_general.csv')

In [10]:
query = """ 
BEGIN
    DECLARE @TotalRows INT;
    DECLARE @SampleSize INT;
    DECLARE @Q1 FLOAT, @Q3 FLOAT, @IQR FLOAT;
    DECLARE @LowerBound FLOAT, @UpperBound FLOAT;

    -- Contar el total de filas
    SELECT @TotalRows = COUNT(*)
    FROM dbo.BI_GAME_SESSIONS gs
    INNER JOIN dbo.BI_CARDS crd ON crd.CARD_ID = gs.CARD_ID
    INNER JOIN dbo.BI_PLAYERS pl ON pl.PLAYER_ID = crd.PLAYER_ID
    WHERE gs.INITIAL_TIME >= '2024-01-01';

    -- Calcular tamaño de muestra
    SET @SampleSize = CEILING(@TotalRows * 0.01);

    -- Calcular Q1 y Q3 usando PERCENTILE_CONT
    WITH OrderedValues AS (
        SELECT CAST(gs.INITIAL_AMOUNT AS FLOAT) AS INITIAL_AMOUNT
        FROM dbo.BI_GAME_SESSIONS gs
        INNER JOIN dbo.BI_CARDS crd ON crd.CARD_ID = gs.CARD_ID
        INNER JOIN dbo.BI_PLAYERS pl ON pl.PLAYER_ID = crd.PLAYER_ID
        WHERE gs.INITIAL_TIME >= '2024-01-01'
    )
  

    -- Seleccionar muestra filtrando outliers
    SELECT TOP (@SampleSize)
           pl.PLAYER_ID,
           pl.DOB,
           gs.AVG_BET,
           gs.INITIAL_AMOUNT,
           gs.GAMES_PLAYED_TOTAL,
           gs.GAMES_WON_TOTAL,
           gs.BET_TOTAL
    FROM dbo.BI_GAME_SESSIONS gs
    INNER JOIN dbo.BI_CARDS crd ON crd.CARD_ID = gs.CARD_ID
    INNER JOIN dbo.BI_PLAYERS pl ON pl.PLAYER_ID = crd.PLAYER_ID
    WHERE gs.INITIAL_TIME >= '2024-01-01'
    ORDER BY CHECKSUM(NEWID());
END
"""

In [11]:
cluster_classifier = joblib.load('../../models/knn_pipeline_foliattiGeneral_v0.pkl')
bet_total_classifier = joblib.load('../../models/bet_total_pipeline_foliattiGeneral_v0.pkl')

c:\Users\ismer\Desktop\Foliatti\venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ismer\Desktop\Foliatti\venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.4.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\ismer\Desktop\Foliatti\venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.4.2 when using versio

In [12]:
load_dotenv()


server = os.getenv('SERVER')
username = os.getenv('SQL_USERNAME')
password = os.getenv('SQL_PASSWORD')

## Neza ##
database = 'ewise_BI_All'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)
Session = sessionmaker(bind=engine)
session = Session()
df = pd.read_sql_query(query, engine)

In [13]:
df['DOB'] = pd.to_datetime(df['DOB'])
hoy = datetime.now()
df['Edad'] = hoy.year - df['DOB'].dt.year
df['Edad'] -= ((hoy.month < df['DOB'].dt.month) | 
               ((hoy.month == df['DOB'].dt.month) & 
                (hoy.day < df['DOB'].dt.day)))

bins = [18, 25, 35, 45, 55, 65, 100]  # 100 es un valor arbitrario para cubrir edades mayores a 65
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']

# Crear la columna 'Rango_Edad'
df['Rango_Edad'] = pd.cut(df['Edad'], bins=bins, labels=labels, right=False)

# le_edad = LabelEncoder()
# df['Rango_Edad_le'] = le_edad.fit_transform(df['Rango_Edad'])

In [14]:
mapping = dict(zip(df_data_general['Rango_Edad'], df_data_general['Rango_Edad_le']))

print(mapping)

df['Rango_Edad_le'] = df['Rango_Edad'].map(mapping)

{'18-24': 0, '45-54': 3, '55-64': 4, '65+': 5, '25-34': 1, '35-44': 2, nan: 6}


In [15]:
def define_bet_total(bet):
    if bet < 500:
        return 0
    elif bet > 500 and bet <= 750:
        return 1
    elif bet > 750 and bet <= 1000:
        return 2
    elif bet > 1000 and bet < 5000:
        return 3
    else:
        return 4
    
df['bet_total_label'] = df['BET_TOTAL'].apply(define_bet_total)

In [16]:
df = df.dropna()

In [17]:
X = df[['AVG_BET','INITIAL_AMOUNT', 'GAMES_PLAYED_TOTAL', 'GAMES_WON_TOTAL', 'Rango_Edad_le']]
y_hat_test = cluster_classifier.predict(X)

In [18]:
df['Cluster'] = y_hat_test

In [19]:
X = df[['Cluster', 'INITIAL_AMOUNT', 'AVG_BET']]
y = df['bet_total_label']
y_hat_bet_total = bet_total_classifier.predict(X)

In [20]:
print(classification_report(y, y_hat_bet_total))

              precision    recall  f1-score   support

           0       0.79      0.96      0.87      7436
           1       0.26      0.13      0.18      1119
           2       0.21      0.07      0.11       676
           3       0.70      0.54      0.61      1912
           4       0.71      0.40      0.51       241

    accuracy                           0.74     11384
   macro avg       0.54      0.42      0.45     11384
weighted avg       0.69      0.74      0.70     11384

